In [1]:
import os
import numpy as np
import sty
import torch
import pprint as pp
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
from tqdm import tqdm
from args_2opt import get_args
from datasets.tsp import TSPDataset
from environments.tsp_2opt import TSP2OPTEnv
from models.tsp_2opt_agent import TSP2OPTAgent

In [2]:
args = get_args('')

args.load_path = "outputs/tsp_100/bi_gru_knn_20210622T021916/epoch-150.pt"
args.val_dataset = "datasets/tsp_100_validation_256.pt"
args.graph_size = 100
args.tour_gnn_layers = 10
args.num_gnn_layers = 5
args.batch_size = 256
args.device = torch.device("cuda" if args.use_cuda else "cpu")

# Pretty print the run args
pp.pprint(vars(args))

# Set the random seed
torch.manual_seed(args.seed)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

{'batch_size': 256,
 'bias': True,
 'checkpoint_epochs': 1,
 'decode_type': 'sampling',
 'decoder_num_heads': 8,
 'device': device(type='cuda'),
 'edge_dim': 1,
 'embed_dim': 128,
 'encoder_num_heads': 8,
 'entropy_beta': 0.005,
 'epoch_size': 5120,
 'epoch_start': 0,
 'eval_batch_size': 1024,
 'eval_only': False,
 'exp_beta': 0.8,
 'gamma': 0.99,
 'graph_size': 100,
 'graph_type': 'knn',
 'horizon': 10,
 'load_path': 'outputs/tsp_100/bi_gru_knn_20210622T021916/epoch-150.pt',
 'log_dir': 'logs',
 'log_step': 10,
 'lr_critic': 0.0001,
 'lr_decay': 1.0,
 'lr_model': 0.0001,
 'max_grad_norm': 0.3,
 'max_num_steps': 200,
 'n_epochs': 200,
 'no_cuda': False,
 'no_norm_return': False,
 'no_progress_bar': False,
 'node_dim': 2,
 'normalization': 'batch',
 'num_gnn_layers': 5,
 'num_workers': 8,
 'output_dir': 'outputs',
 'pooling_method': 'mean',
 'problem': 'tsp',
 'run_name': 'run_20210624T172215',
 'save_dir': 'outputs/tsp_50/run_20210624T172215',
 'seed': 1234,
 'tanh_clipping': 10.0,
 't

In [3]:
load_data = torch.load(args.load_path)
env = TSP2OPTEnv()
model = TSP2OPTAgent(args).to(args.device)

model.load_state_dict(load_data["model"])

<All keys matched successfully>

In [4]:
val_dataset = TSPDataset(
    size=args.val_size, graph_size=args.graph_size, graph_type=args.graph_type,
    load_path=args.val_dataset
)

In [23]:
model.train()
for bat in DataLoader(val_dataset, batch_size=args.eval_batch_size):
    step = 0
    bat = bat.to(args.device)
    with torch.no_grad():
        node_pos = to_dense_batch(bat.pos, bat.batch)[0]
        done = False
        state = env.reset(T=200, node_pos=node_pos)
        embed_data = model.init_embed(bat)
        node_embeddings, _ = model.encoder(embed_data)
        while not done:
            step += 1
            action, _, _ = model(state, node_embeddings, embed_data.batch)
            state, _, done, _ = env.step(action.squeeze())
            if(step%100==0):
                avg_len = state.best_tour_len.mean().item()
                opt = sum([g.opt for g in val_dataset]) / len(val_dataset)
                opt_gap = (avg_len - opt) / opt
                print(f"step:{step}, average length:{avg_len:.3f}, gap:{opt_gap*100:.3f}%")


step:100, average length:11.075, gap:42.360%
step:200, average length:8.518, gap:9.485%
step:300, average length:8.316, gap:6.887%
step:400, average length:8.245, gap:5.973%
step:500, average length:8.202, gap:5.430%
step:600, average length:8.179, gap:5.128%
step:700, average length:8.159, gap:4.872%
step:800, average length:8.142, gap:4.651%
step:900, average length:8.129, gap:4.487%
step:1000, average length:8.116, gap:4.320%
step:1100, average length:8.109, gap:4.228%
step:1200, average length:8.101, gap:4.123%
step:1300, average length:8.094, gap:4.039%
step:1400, average length:8.090, gap:3.987%
step:1500, average length:8.086, gap:3.941%
step:1600, average length:8.082, gap:3.889%
step:1700, average length:8.080, gap:3.854%
step:1800, average length:8.074, gap:3.782%
step:1900, average length:8.066, gap:3.675%


In [19]:
state.best_tour_len.mean()

tensor(8.5639, device='cuda:0')